Mapping our Twitter followers
------------------------------

![Bot](https://devumi.com/wp-content/uploads/2014/11/Devumi-Services-BG2.jpg)

As I write this, our account [@RosieTuring](https://twitter.com/RosieTuring) has 5,918 followers (despite the fact that we only purchased 5,000. (How can that be? Sloppy bot deployment? Bot affinity? Maybe real people following along? How did they find us?) Recall that we first bought 2,500 from [Devumi](https://devumi.com) and then another 2,500 from [Tw-Growth](http://www.tw-growth.com/). We selected the first company by a simple Google search and their rank on the [Buy Followers Guide](http://buyfollowersguide.com/). 

**Detour**

Since we are now backing into the topic of seeing how networks can amplify the "force" of information, Lawrence Alexander has [a  blog post on how to uncover connections between web sites](https://www.bellingcat.com/resources/how-tos/2015/07/23/unveiling-hidden-connections-with-google-analytics-ids/). Here's how he sets up the problem he  explores.

> Imagine you are researching an anonymous website with no clues as to its authorship. A single string of code could link that site to another, which perhaps reveals the identity of the owner. With the right tools, this information can be surprisingly easy to uncover.

These strategies will come in handy when we examine the spread of misinformation among groups of "news" sites. To give you a taste, using the site [SpyOnWeb](http://www.spyonweb.com/) (suggested in Alexander's blog post), we can "easily disclose websites with the same ip address, advertising and statistic codes." So, starting with devumi.com, we find it's site kin. [Have a look.](http://www.spyonweb.com/devumi.com) You see competing Twitter bot sites as well as other  review sites sharing Google Analytics IDs -- the same organizataion is monitoring traffic across this group of sites, suggesting a connection. This is a detour, but we wanted to give you a sense that networks live throughout the technologies we are examining for this class.

**Our followers**


First, remember the resouces from the end of last class (+1 from Kat).

[twitteraudit](https://www.twitteraudit.com/)

[Fake Follower Check](https://fakers.statuspeople.com/)

[Bot or Not](http://botornot.co/)

[Why can't Twitter kill its bots](http://fusion.net/story/195901/twitter-bots-spam-detection/)

[The Rise of Social Bots](http://cacm.acm.org/magazines/2016/7/204021-the-rise-of-social-bots/fulltext)

[The DARPA Twitter Bot Challenge](https://arxiv.org/abs/1601.05140)

[How Twitter Bots Are Shaping the Election](https://www.theatlantic.com/technology/archive/2016/11/election-bots/506072/)

[The ‘Star Wars’ botnet with >350k Twitter bots](https://arxiv.org/pdf/1701.02405.pdf)

We're now going to use some of these teachings together with our own research to spot our fake followers.

In class last time, we saw how the Twitter API could be used to pull information about users of the system. In particular, we had a look at the `followers/ids` API that lets you scoop up 5,000 followers at a time. That's just a little short of our account's total. To make things easier, we have prepared a JSON object that you can load into Python -- when we translate the JSON into Python, we'll see that it's a list of user IDs, one entry for each of our followers. 

The JSON object you need is part of a bigger set of data that we have prepared for you (and that you will take some time to recreate). You can [download the file here](http://compute-cuj.org/rosie.tar.gz), uncompress it and move it to the folder where the notebook is located. 

The command below reads in the JSON object representing our follower list, and produces an actual Python list of ids.

In [ ]:
from json import loads

followers = loads(open("rosie/rosie_follower_ids.json").read())

In [ ]:
type(followers)

"followers" is a list, as expected. And it's length?

In [ ]:
len(followers)

It matches what we expect -- 5,918 followers.

The ids in this list are sorted in the order the accounts followed us, with the most recent followers coming first and the oldest followers at the end of the list. We know some of Rosie's followers because members of the class jumped right in to follow her a couple hours after we made the purchase with Devumi. In other words, your classmates should be early followers, so among the last entries in the "followers" list. Using the web site [tweeterid.com](https://tweeterid.com/), we can translate between twitter handles and ids. For example, Manuel has id 704397770325622784. Is he among our followers?

In [ ]:
704397770325622784  in followers

And what position in the list? The larger the number, the earlier he followed us.

In [ ]:
followers.index(704397770325622784)

This means he is in position 5854 in our list of 5,918 followers. One of the first 100! Let's check a nearby entry. If Manuel's was 5854, let's look at 5855. 

In [ ]:
followers[5855]

Who belongs to the index 516093085? Is this another student in the class? Use [tweeterid.com](https://tweeterid.com/) to find out. Now, let's widen the net and see if there are any other classmates who followed Rosie at about the same time. Here's how we form the sublist that focuses on entries 5850 through 5859 of followers.

In [ ]:
followers[5850:5860]

We'll focus on these ids for a moment.

**Using the Twitter API**

Notice that the Twitter ids come in various sizes -- some have 10 digits, some have many more. Why? Now, we can (just so we don't forget) use the Twitter API to pull information on ids. Copy your keys from the last notebook and put them below. We will then use Tweepy again and create a variable "auth" and a variable "api" -- the former holding our authentication information, and the latter encapsulating the operations of the API. 

In [ ]:
CONSUMER_KEY = "your key"
CONSUMER_SECRET = "your secret"
ACCESS_TOKEN = "your access token"
ACCESS_TOKEN_SECRET = "your access secret"

In [ ]:
from tweepy import OAuthHandler, API

# setup your authentication
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# create an object we will use to communicate with the Twitter API
api = API(auth)
type(api)

We can now pull profiles from the API for any id. The last one in the slice above is 451891461. We will use the  `users/show` API: [https://dev.twitter.com/rest/reference/get/users/show](https://dev.twitter.com/rest/reference/get/users/show). Recall that this call returns a "User" object that contains data about the user (their screen name, number of followers and so on) as well as methods like follow() and unfollow(). Here we pull the data for single user  from the list above.

In [ ]:
# get a user's profile (the 'nytimes' in this case)
user = api.get_user(user_id=451891461)

# print out some of the user's information
print user.screen_name
print user.statuses_count
print user.friends_count
print user.followers_count
print user.description

Are they a bot? Legit? What do you think? Now, replace the id 451891461 in the code above with another one from the list and see who it is and what their description is. 

We are now going to run through the list systematically and pull the same information, and print it out. We've created a loop that uses the variable "id" to take on ids in the follower list at index 5850, 5851 and so on. We then issue the get_user() API call and print. Do you see other classmates in our list? (Remember, we had a moment when we unveiled the account and people just followed Rosie. So there's good reason to see a number of classmates close together in this list.)

In [ ]:
for id in followers[5850:5860]:

    user = api.get_user(user_id=id)

    # print out some of the user's information
    print user.screen_name
    print user.statuses_count
    print user.friends_count
    print user.followers_count
    print user.description
    print "--"*10

The get_user() API can be called either with the argument "screen_name" or with "user_id" as we did above. Below we pull the data from Rosie's account and remind ourselves that the result of the API call is a "User" object. It contains data that we access as attributes of the object. Like ".screen_name". 

In [ ]:
user = api.get_user(screen_name="RosieTuring")

In [ ]:
print type(user)

In [ ]:
user.screen_name

This User object is really handy. Encapsulating data and functions  in this way is the soul of object-oriented programming. We want to create (and use!) objects that nicely represent Users or Profiles or Timelines. So Tweepy is helping us out considerably, hiding a lot of complexity. Tweepy's objects let us interactd with the Twitter API at a high level, without worrying about what goes on in the background. We just get_user().

**Back to JSON**

The result of a call to the Twitter API, however, is technically a JSON string. Tweepy takes that string and wraps into a User object, say. But, as we did with tweets, we could instead parse it into primitive Python objects like lists and dictionaries and numbers and strings. The original JSON, translated to primitive data types is stored in the ".\_json" attribute of a User object.

In [ ]:
rawuser = user._json
type(rawuser)

So we have a dictionary. What are its keys?

In [ ]:
rawuser.keys()

And let's drill into the object. We can select the user's screen_name or their followers_counts by looking up these keys in the dictionary. 

In [ ]:
rawuser["screen_name"]

In [ ]:
rawuser["followers_count"]

I belabor this point because rather than have you hit the Twitter API 5,918 times, we have pulled all of Rosie's followers' profile data. They are stored in a folder that you have already downloaded. It is called `rosie/friends`. So, let's take the 20th entry from the back of the "followers" list. Remember, negative indices count from the back of the list. So -1 is the last entry, -2 is the second to the last and so on. 

In [ ]:
followers[-20]

The JSON profile for the user with id 2437830887 is stored in `rosie/users/2437830887.json`. In the cell below, we create the filename, then open it and read the JSON. We turn the string into Python primitives.

In [ ]:
filename = "rosie/users/"+str(2437830887)+".json"

rawuser = loads(open(filename).read())
rawuser

So again, we have a dictionary and we can dig into it providing keys. Like, say, "status", the user's last tweet. It's again a dictionary that has keys related to when the tweet was created and so on.

In [ ]:
rawuser["status"]

OK so let's now print out information from the last 50 users in the followers list. That is, our first 50 followers (remembering ids are sorted in reverse chronological order). So here we create filenames in a loop and print out the screen names. 

Remember we can subset the list "followers" to extract certain ids in a number of ways. The "slice" followers[5850:5860] gives us ids with indices 5850 through 5859. The selection followers[30] gives you the id with index 30 (or the 31st element since we start counting at 0). The slice followers[-20] gives you the 20th from the end of the list. The slice followers[-10:-2] includes the 10th from the end, up to the 3rd from the end. Finally, you can leave off numbers on either side of the ":" to indicate you want the slice to run from the beginning of the list as in followers[:10], or to the end of the list as in followers[-50:]. And, for completeness, the simple expresson followers[:] returns the whole list. 

Below we want the first 50 followers to follow Rosie, or the last 50 elements of our list.

In [ ]:
for id in followers[-50:]:
    
    filename = "rosie/users/"+str(id)+".json"
    rawuser = loads(open(filename).read())
    
    print rawuser["screen_name"]

We've sat with these bots for some time now and noticed that they tend to be lifted directly from existing Twitter users. Their "name" and "location" are copied, as are any images. Of course the screen name has to change and this, so far, Devumi seems to follow some simple rules. I've seen two techniques for transforming a real person's screen name into a robot's.

* A lowercase "i" is replaced for a lowercase "l" or vice versa,
* A 0 (zero) is replaced for an "o", and

There are no doubt other rules that we should try to document. 

**1. We want to come up with rules about when our followers are bots, and in the case of Devumi's bots, link them to actual people. To start, we will focus on just the screen names of the bots. From the last 50 ids in the followers list (printed out above), try to link them to real people. Can you spot a bot just by their name? Can you identify the real user whose identity was coopted for a bot, just from the screen name? Add to my list of techniques for making screen names above. Finally, print out another 50 from the somewhere in the last 2,500 entries of the followers list (as these are our first followers and probably Devumi bots) and repeat this process. What do you find?***

Now, let's venture beyond simple screen_names. What other clues might we find from the user profiles that we are looking at a Devumi bot? Here is the same code from the cell above, looping over the last 50 followers in the list. Only now we printout the screen name and whether the tweet was a "native" retweet (um, hitting a retweet button). We have probably observed that our bots do very little original tweeting but only retweet. Let's see how many retweet natively...

In [ ]:
for id in followers[-50:]:
    
    filename = "rosie/users/"+str(id)+".json"
    rawuser = loads(open(filename).read())
    
    print rawuser["screen_name"], rawuser["status"]["retweeted"]
    print "--"*10

So, our bots are not "native" retweeting. That means a retweet must involve an "RT" in the tweet text. For example, the last tweet from Rosie is a retweet of Kat. Here we grab the last two tweets and have a look at the text and wheter the tweet object marked it as a retweet.

In [ ]:
tweets = api.user_timeline(screen_name='rosieturing', count=2)
tweets[0].text

In [ ]:
tweets[0].retweeted

Now, going back to our followers list (rather than calling the API), let's repeat our loop but this time look at the first 10 characters of each tweet and see what we have. Do they start "RT"?

In [ ]:
for id in followers[-50:]:
    
    filename = "rosie/users/"+str(id)+".json"
    rawuser = loads(open(filename).read())
    
    print rawuser["screen_name"], rawuser["status"]["text"][:10]
    print "--"*10

So we see lots of RT's at the beginning of the tweets. We also see tweets without RT's, meaning they are actual tweets. Were the accounts that are missing the RT's among those you would have labeled as real and not a robot? Let's now dig into the different aspects of these users and see if we have other clues besides their names as to whether they are bots (or not). You've done a fair bit of reading so far, so try out the measures of botness suggested in the papers and the bot-or-not web sites you've seen.

**2. Use your observations from Question 1 and apply them to another chunk of our followers. Work with slices that are subsets of "-2500:", as the first 2,500 that followed us in time are the last 2,500 in the followers list and are more likely part of the batch of bots we bought from Devumi.  Then, try to come up with a procedure for spotting our bots, including the name change hints you found in Question 1, but expanding to other kinds of data from the user's profile. Use the API if you need information about the user that is not included in the profile (say, their last 10 tweets). For the Devumi bots, is their botness obvious?**

**3. Our followers numbers keep floating around. It would be interesting to know if any of these accounts are being shutdown. Check a few of our older Devumi followers (using the Twitter API) and see.**

**4. Next, we want to look at the tw-growth bots. I have done absolutely nothing with them. They should be safely in the range from 5,000 to 2,500, although the Devumi bots kept adding slowly even though we had 2,500 of them. Look at these bots and use some of the same tactics you followed for Devumi, coming up with logic about how you might spot these bots. Do these seem associated with real people?**

**Aside**

Keith Collins, an alum, posted a piece on Donald Trump's tweets called ["An interactive guide to Donald Trump’s Twitter exclamations. Enjoy!"](https://qz.com/908392/each-exclamation-that-donald-trump-has-ended-a-tweet-with-in-the-past-year-wow/) In principle, you have all you need to complete this project, albeit with Plotly. 

Think about how you might do it!

In [ ]:
# get the "real" Donald's last 200 tweets
tweets = api.user_timeline(screen_name='realDonaldTrump', count=200)

for tweet in tweets:
    print tweet.text

**Networks**

Now, let's consider the relationships between these bots. How does Devumi structure their network? Our bots follow a lot of people and how are new clients like ourselves allocated bots? Do they have 2,500 bots that service everyone? Are there more? Who follows them? Who do they follow?

To help answer this, we provided more files (although we are only up to 1,800 of the 5,918 so far due to Twitter rate limiting) that are named after a follower of Rosie's and contains a list of ids **they follow**. Twitter calls these **friends**. We will later look at who follows our followers, although this is a much smaller number. We will also look at who they retweet to see which clients paid Devumi for better service.

First, let's create a dataframe of "follow" relationships. We will scoop up each file we have, and then create a data frame where each row is one of Rosie's followers following another id -- one of Rosie's followers and their friends. Does that make sense? We have put comments in the code below to help you see what's going on.

One of the great things about Python is that you can easily loop over files or folders and process data. Here we look at the `rosie/friends` folder and listout all the `.json` files. Their name encodes the id of one of Rosie's followers and the contents is a JSON list of the id's that user follows, their friends. 

So, look at `rosie/friends/143203718.json` (open it in a text editor of some kind, say). You'll see it's a list of long numbers, Twitter ids. The name of the file `143203718.json` means that it contains all the ids of users that 143203718 follows -- all of 143203718's friends. OK? Filename encodes one of Rosie's followers (143203718) and list content describes their friends.

This code will take a little while.

In [ ]:
# import the ability to read JSON and also to 
# list the contents of a folder (a directory) on your computer 

from os import listdir
from json import loads

# create two lists, one for the id of rosie's followers and one for
# the ids of people they follow

rosie_followers = []
their_friends = []

path = "rosie/friends/"

# loop over all the files in our "friends" folder

for follower_json_file in listdir(path):
    
    # give the location of the file 
    filename = path+follower_json_file
    
    # use the file name, and highlight the id Rosie's follower
    # we use int() to turn the string into an integer. oh and we
    # strip off the ".json".
    follower_id = int(follower_json_file.replace(".json",""))

    # open the file and load the json string into python
    friends_for_one_follower = loads(open(filename).read())
    
    # add the data for this user's friends to our data set
    their_friends = their_friends + friends_for_one_follower
    rosie_followers = rosie_followers + [follower_id]*len(friends_for_one_follower)
    
    # the last line creates a list of the current follwer_id that is as
    # long as the list of that id's friends list. We are just repeating
    # the follower id as many times as the id has friends so we can line
    # up the two columns.

In [ ]:
len(rosie_followers)

So Rosie's followers follow 2.9M users. We will now take the two lists and turn them into columns of a Data Frame. We will use column names "From Name" and "To Name" to match what Graph Commons wants for its data format. From now on the code runs quickly.

In [ ]:
from pandas import Series, DataFrame
from numpy import where

df = DataFrame({"Rosie follower":rosie_followers,"their friend":their_friends})

In [ ]:
df.head()

With this structure, we can how see how many people our bots are friends with. The command value_counts() continues to be infinitely useful. 

In [ ]:
df["Rosie follower"].value_counts().head(20)

Now, summarize the people Rosie's followers follow with a value_counts(). There should be an entry for Rosie at the top, as they all follow her, and then things should drop off. What do you see? What does it mean?

In [ ]:
# put your code here
df["their friend"].value_counts().head(20)

So, some of the friends are followed by a large number of the accounts that also follow us. id 15097615, for example, is followed by 1654 of the 1810 bots we have pulled data for so far. So let's look at the overlap between these high scoring accounts.

Toward this end, we can make two subsets, one corresponding to all of Rosie's followers who also follow 15097615 (are friends with 15097615) and one for all of Rosie's followers who are friends with 1650547956. Is there any overlap? 

In [ ]:
one = df[df["their friend"]==15097615]
one.shape

In [ ]:
two = df[df["their friend"]==1650547956]
two.shape

We can now use .isin() to tell us how many of our followers that also follow 15097615 also follow 1650547956. Remember that .isin() returns a Boolean that we can then sum() to get the number of True values...

In [ ]:
two["Rosie follower"].isin(one["Rosie follower"]).head(5)

In [ ]:
sum(two["Rosie follower"].isin(one["Rosie follower"]))

That's a lot of overlap. For id 15097615, it means that it has at least 1654 Devumi bots and 1437 of them, or just over 85%, are identical to those following 1650547956. So let's  make this clear. The vast majority of our 1810 data files that we have so far represent bots. These 1810 bots follow us, but they also follow other people. In particular, 1654 of them also follow id 15097615 and 1544 of them follow id  1650547956. The catch is that a large fraction of the same bots also follow both of these ids. What it says is that some bots are deployed to follow lots of the same people. 

To get at this effect in a more systematic way, we'll start to look at so called networks of relationships. So we've started mapping out pieces of the network in the last paragraph, but we can do it more systematically.

Here we create a subset of the data, consisting of just those rows where friends are followed by between 100 and 200 of our bots. This gives us a manageable network. Here is the code. Everything should be familiar, except perhaps the .sample() command. It just takes a sample of the friends ids, rather than using all of them (there are thousands).

In [ ]:
tmp = df["their friend"].value_counts()
tmp = tmp[(tmp<200) & (tmp>100)].sample(30)

Then we use .isin() again to keep just the rows in our data frame that have one of the 30 friends we want. 

In [ ]:
dfs = df[df["their friend"].isin(tmp.index)]
dfs.head(5)

To prepare thie data frame for input to Graph Commons, we have to change the column names. Rosie follower will become "From Name" and the friends of her followers will be stored under "To Name". This indicates the follower relationship here. One of our followers follows somone else.

We also insert a row of all "rosie followers" called "From Type" because Graph Commons will distinguish between our bots (followers) and their friends. The insert() command wants the index of the column to insert the new data in front of, the name of the new column and then data for the new column. Here we just give a string "rosie follower" that will be repeated to fill out the new column.

In [ ]:
dfs.columns = ["From Name","To Name"]
dfs.insert(0,"From Type","rosie follower")

In [ ]:
dfs.head(5)

Next, insert a column representing the "To type". Most of these will be friends, but a few of them are also our followers. That is, in 32 cases, one bot follows another bot. And we want to make sure we highlight that connection right. So we use the command where() that takes a boolean expression and returns, in this case, "rosie follower" if the condition is True and "friend" otherwise. 

In [ ]:
dfs.insert(2,"To Type", where(dfs["To Name"].isin(dfs["From Name"]),"rosie follower","friend"))

In [ ]:
dfs.head(5)

Finally, assign the kind of relationship we have between From and To. In this case it's "follows".

In [ ]:
dfs.insert(2,"Edge","follows")
dfs.head(5)

Now, write out the data as a CSV, walk over to [Graph Commons](https://graphcommons.com) site and "import" a new data set. You will have to register, but it's free.

In [ ]:
dfs.to_csv("edges.csv",index=False)

![network](http://compute-cuj.org/GraphCommons-2017-2-12-10-31-49.png)

The edges.csv is all you need. You should get an image sort of like this one. [The actual graph is hosted at Graph Commons](https://graphcommons.com/graphs/70ce7bb9-99aa-4e72-9878-fcae25428305). Explore! How does it compare to yours? You can clearly see overlap, you can clearly see situations when the bots follow only one of our 30 friends. This is just the beginning!
 
For the next time, we will continue with the Bot or Not work as well as refine this network display to tell us more about how our bots are mobilized.

**Bonus** 

Try building one on your own! The only constraint is that you should have less that 4500 rows in your edges file. Otherwise you have to break it up into multiple 4500 row chunks. OK? What kind of connectivity would you like to look at? You have the file format... Go!